In [ ]:
!pip install selenium

In [ ]:
!pip install pandas openpyxl

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


USER_AGENT environment variable not set, consider setting it to identify your requests.


## 공지사항 크롤링(10개)
- 공지사항에는 댓글이 없음
- 이미지도 url과 함께 가져오기

In [3]:
# 이미지 같이 불러오기
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get('https://forum.netmarble.com/enn_ko/list/3/1')
notice = []

for i in range(1, 11):
    next_path = f'//*[@id="articleListSubView"]/li[{i}]/div[2]/div[2]/div[1]/a'
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_path))).click()
    
    b_path = '//*[@id="appView"]/div[1]/div[2]'
    content_ = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, b_path)))

    # 이미지 URL 로드
    try:
        b_path = '//*[@id="contentsDetail"]'
        img_elements = driver.find_elements(By.XPATH, f'{b_path}//img')  # 내용 내부의 모든 이미지 요소 찾기
        img_urls = [img.get_attribute('src') for img in img_elements]
        img_urls_str = '; '.join(img_urls)  # 여러 이미지 URL을 문자열로 결합
    except TimeoutException:
        img_urls_str = '이미지 없음'

    notice.append({'notice': content_.text, 'image_urls': img_urls_str})
    
    driver.back()
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, next_path)))
    time.sleep(1)  

driver.quit()


In [4]:
# DataFrame으로 변환
df = pd.DataFrame(notice)

# Excel 파일로 저장
df.to_excel('C:/Users/jyp/.conda/envs/langchain-cource/00_PT/notice_with_image.xlsx', index=False, engine='openpyxl')

print("Excel 파일이 성공적으로 저장되었습니다.")

Excel 파일이 성공적으로 저장되었습니다.


## 업데이트 (10개)
- 댓글 없음
- 이미지 같이 저장

In [5]:
# 이미지 같이 불러오기
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get('https://forum.netmarble.com/enn_ko/list/21/1')
notice = []

for i in range(1, 11):
    next_path = f'//*[@id="articleListSubView"]/li[{i}]/div[2]/div[2]/div[1]/a'
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_path))).click()
    
    b_path = '//*[@id="appView"]/div[1]/div[2]'
    content_ = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, b_path)))

    # 이미지 URL 로드
    try:
        b_path = '//*[@id="contentsDetail"]'
        img_elements = driver.find_elements(By.XPATH, f'{b_path}//img')  # 내용 내부의 모든 이미지 요소 찾기
        img_urls = [img.get_attribute('src') for img in img_elements]
        img_urls_str = '; '.join(img_urls)  # 여러 이미지 URL을 문자열로 결합
    except TimeoutException:
        img_urls_str = '이미지 없음'

    notice.append({'notice': content_.text, 'image_urls': img_urls_str})
    
    driver.back()
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, next_path)))
    time.sleep(1)  

driver.quit()


In [6]:
# DataFrame으로 변환
df = pd.DataFrame(notice)

# Excel 파일로 저장
df.to_excel('C:/Users/jyp/.conda/envs/langchain-cource/00_PT/update_with_image.xlsx', index=False, engine='openpyxl')

print("Excel 파일이 성공적으로 저장되었습니다.")

Excel 파일이 성공적으로 저장되었습니다.


## 공략 & TIP (10개)
- 댓글 있음
- 이미지도 같이

In [7]:
# 이미지 같이 불러오기
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Selenium 설정
driver = webdriver.Chrome()
driver.get('https://forum.netmarble.com/enn_ko/list/27/1')
tip = []

for i in range(1, 11):  # 예시로 10개의 크롤링
    next_path = f'//*[@id="articleListSubView"]/li[{i}]/div[2]/div[2]/div[1]/a'
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_path))).click()
    
    #  내용 로드
    b_path = '//*[@id="appView"]/div[1]/div[2]'
    content_ = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, b_path)))
    
    # 이미지 URL 로드
    try:
        b_path = '//*[@id="contentsDetail"]'
        img_elements = driver.find_elements(By.XPATH, f'{b_path}//img')  # 내용 내부의 모든 이미지 요소 찾기
        img_urls = [img.get_attribute('src') for img in img_elements]
        img_urls_str = '; '.join(img_urls)  # 여러 이미지 URL을 문자열로 결합
    except TimeoutException:
        img_urls_str = '이미지 없음'
    
    try:
        # 댓글 영역 로드 시도
        rep_path = '//*[@id="replyFilterArea"]/div'
        rep = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, rep_path)))
        
        # 불필요한 부분 제거
        full_text = rep.text
        filtered_lines = [line for line in full_text.split('\n') if line not in ['최신순', '등록순', '답글순', '필터', '답글','모두 보기']]
        rep_filtered = '\n'.join(filtered_lines)
        
    except TimeoutException:
        # 댓글 영역이 없는 경우
        rep_filtered = '댓글 없음'
    
    # 결과 저장
    tip.append({'content': content_.text, 'reply': rep_filtered, 'image_urls': img_urls_str})

    # 뒤로 가기
    driver.back()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, next_path)))

driver.quit()


In [8]:

# DataFrame으로 변환
df = pd.DataFrame(tip)

# Excel 파일로 저장
df.to_excel('C:/Users/jyp/.conda/envs/langchain-cource/00_PT/tip_with_images.xlsx', index=False, engine='openpyxl')

print("Excel 파일이 성공적으로 저장되었습니다.")


Excel 파일이 성공적으로 저장되었습니다.


## 자유게시판 (30개)
- 댓글 있음
- 이미지 불러오기

In [11]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

# Selenium 설정
driver = webdriver.Chrome()
driver.get('https://forum.netmarble.com/enn_ko/list/33/1')
board = []

for page in range(2):
    for i in range(1, 16):  
        next_path = f'//*[@id="articleListSubView"]/li[{i}]/div[2]/div[2]/div[1]/a'
        
        try:
            # 요소를 다시 찾기
            element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_path)))
            element.click()

            b_path = '//*[@id="appView"]/div[1]/div[2]'
            content_ = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, b_path)))

            try:
                b_path = '//*[@id="contentsDetail"]'
                img_elements = driver.find_elements(By.XPATH, f'{b_path}//img')  # 내용 내부의 모든 이미지 요소 찾기
                img_urls = [img.get_attribute('src') for img in img_elements]
                img_urls_str = '; '.join(img_urls)  # 여러 이미지 URL을 문자열로 결합
            except TimeoutException:
                img_urls_str = '이미지 없음'

            try:
                # 댓글 영역 로드 시도
                rep_path = '//*[@id="replyFilterArea"]/div'
                rep = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, rep_path)))

                # 불필요한 부분 제거
                full_text = rep.text
                filtered_lines = [line for line in full_text.split('\n') if line not in ['최신순', '등록순', '답글순', '필터', '답글','모두 보기']]
                rep_filtered = '\n'.join(filtered_lines)

            except TimeoutException:
                # 댓글 영역이 없는 경우
                rep_filtered = '댓글 없음'

            # 결과 저장
            board.append({'content': content_.text, 'reply': rep_filtered, 'image_urls': img_urls_str})

            # 뒤로 가기
            driver.back()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, next_path)))
        
        except (TimeoutException, StaleElementReferenceException):
            print(f"Failed to process article {i} on page {page}")
            continue
    
    try:
        next_page_xpath = '//*[@id="appView"]/div[2]/div/div[2]/div[3]/div[2]/div/ul/li[2]/a'  
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_page_xpath))).click()
    except TimeoutException:
        print(f"Failed to navigate to page {page + 1}")
        break

driver.quit()


Excel 파일이 성공적으로 저장되었습니다.


In [12]:

# DataFrame으로 변환
df = pd.DataFrame(board)

# Excel 파일로 저장
df.to_excel('C:/Users/jyp/.conda/envs/langchain-cource/00_PT/board_with_images.xlsx', index=False, engine='openpyxl')

print("Excel 파일이 성공적으로 저장되었습니다.")


Excel 파일이 성공적으로 저장되었습니다.
